<h1 align=center><font size=5>Recommender System for Visiters to New York City Using Foursquare Data</font></h1>

### Data Preparation 1: Geological
The beginning of our code will follow the lab. It will get us a dataframe for venue locations in nyc.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library
!pip install spacy
import spacy

print('Libraries imported.')

     |████████████████████████████████| 92kB 15.3MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 8.1MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 43.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 48.6MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 44.9MB/s eta 0:00:01
     |████████████████████████████████| 194kB 42.9MB/s eta 0:00:01
  Found existing installation: importlib-metadata 0.7
    Uninstalling importlib-metadata-0.7:
      Successfully uninstalled importlib-metadata-0.7
Libraries imported.


In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [108]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [109]:
neighborhoods_data = newyork_data['features']

In [110]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [111]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [112]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [92]:
CLIENT_ID = 'QQTY4VLZGKB1XJFTYV1A5X0K11UMGFZRWW5SIRYMN1LSNXPX' # your Foursquare ID
CLIENT_SECRET = 'B2ZUZ0DFKCTLCXGTK12R03GDCZLAU4VP2B3Y0ZIVVZKIF14D' # your Foursquare Secret
VERSION = '20200216' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QQTY4VLZGKB1XJFTYV1A5X0K11UMGFZRWW5SIRYMN1LSNXPX
CLIENT_SECRET:B2ZUZ0DFKCTLCXGTK12R03GDCZLAU4VP2B3Y0ZIVVZKIF14D


In [11]:
LIMIT = 10
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [15]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898083,-73.850259,Caribbean Restaurant


In [16]:
borough_dict = dict(zip(neighborhoods['Neighborhood'],neighborhoods['Borough']))

In [17]:
nyc_venues['Borough'] = pd.Series([borough_dict[N] for N in nyc_venues['Neighborhood']])

In [18]:
nyc_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop,Bronx
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy,Bronx
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop,Bronx
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station,Bronx
4,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898083,-73.850259,Caribbean Restaurant,Bronx


By now, we have obtained the nyc venues data, listing: **neighborhood, latitude, longitude, venue name, venue latitude, venue category and borough.**

## Data Preparation 2: User & Venue

In [21]:
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 

In [54]:
def venues_in_category(cate, lat, long):
    """
    Finding all venues in category
    """
    search_query = cate
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format( \
        CLIENT_ID, CLIENT_SECRET, lat, long, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    
    venues = results['response']['venues']
    return json_normalize(venues)

In [23]:
def venue_rating(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 0
    return rating

In [24]:
def venue_tips(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    try:
        tips = result['response']['venue']['tips']['count']
    except:
        tips = 0
    return tips

# Start Analyzing (Example) : Calculation of scores

We give an example. 

In [74]:
req = {"Borough":"Manhattan", "Neighborhood": "None", "Features":["restaurant", "art"], "Reclusiveness":"busy", "Rating higher than": 7.0, "Color by":"relevance"}        

In [75]:
borough = req['Borough']
neigh = req['Neighborhood']
feats = req['Features']
rec = req['Reclusiveness']
rat = req['Rating higher than']
color_by = req['Color by']

In [27]:
def get_address(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    
    location = geolocator.geocode(address)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except:
        return None
    return (latitude, longitude)

We first get the latitude and longitude of the borough being required.

In [59]:
manhattan_venues = nyc_venues[nyc_venues['Borough'] == "Manhattan"]

We then get the categories relevant to the features.

In [28]:
# use spacy to filter relevants
!python -m spacy download en

     |████████████████████████████████| 12.0MB 1.6MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.tmp/pip-ephem-wheel-cache-vsp1pk27/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/envs/Python36/lib/python3.6/site-packages/en_core_web_sm -->
/opt/conda/envs/Python36/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [29]:
nlp = spacy.load('en')

In [76]:
nyc_venues.to_csv("nyc_venues.csv")

newyork_data.json  nyc_venues.csv


In [30]:
nyc_venues['Category Vector'] = [nlp(cate).vector for cate in nyc_venues['Venue Category']]

In [31]:
loc_dict = dict(zip(nyc_venues['Neighborhood'], zip(nyc_venues['Neighborhood Latitude'], nyc_venues['Neighborhood Longitude'])))

In [32]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [58]:
def find_relevant_venues(neighborhood):
    data = nyc_venues[nyc_venues['Neighborhood'] == neighborhood].reset_index()
    cates_vectors = data['Category Vector']
    relevants = []
    for feat in feats:
        feat_vec = nlp(feat).vector
        for i, vec in enumerate(cates_vectors):
            sim = cosine_similarity(feat_vec, cates_vectors[i])
            if sim > 0.4:
                relevants.append(data['Venue Category'][i])
    relevants = list(set(relevants))
    lat, long = loc_dict[neighborhood]
    
    venues = []
    for cat in relevants:
        try:
            venue = list(venues_in_category(cat, lat, long)['id'])
            venues.extend(venue)
        except:
            continue
    return venues

We get venue counts, venue ratings, and tip counts for all relevant venues for each neighborhood.

In [93]:
neighborhoods = nyc_venues[nyc_venues['Borough'] == borough]['Neighborhood'].unique()

venue_counts = []
ratings = []
tip_counts = []


for neigh in neighborhoods:
    venues = find_relevant_venues(neigh)
    venue_counts.append(len(venues))
    rating = []
    tip = []
    for venue in venues:
        rt = venue_rating(venue)
        if rt > rat and rt != 0:
            rating.append(rt)
        tp = venue_tips(venue)
        if tp != 0:
            tip.append(tp)
    ratings.append(np.mean(rating))
    tip_counts.append(np.sum(tip))

/opt/conda/envs/Python36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/envs/Python36/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [97]:
relevance_score = (np.array(venue_counts) - np.min(venue_counts))/ (np.max(venue_counts) - np.min(venue_counts))
rating_score = (np.array(ratings) - np.min(ratings))/ (np.max(ratings) - np.min(ratings))
reclusive_score = (np.array(tip_counts) - np.min(tip_counts))/ (np.max(tip_counts) - np.min(tip_counts)) if (rec == "busy") \
    else (1 - (np.array(tip_counts) - np.min(tip_counts))/ (np.max(tip_counts) - np.min(tip_counts)))

/opt/conda/envs/Python36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


## Visualization: The Higher the Score, the Deeper the Color

In [139]:
import folium
latitude, longitude = get_address(borough)
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)


original_score = relevance_score if color_by == "relevance" else rating_score
final_score = (0.7*original_score + 0.3*reclusive_score)
final_score = (final_score  - np.min(final_score)) / (np.max(final_score) - np.min(final_score))

# add markers to map
neigh_ = []
for lat, lng, borough, neighborhood in zip(nyc_venues['Neighborhood Latitude'], nyc_venues['Neighborhood Longitude'], nyc_venues['Borough'], nyc_venues['Neighborhood']):
    if neighborhood in neighborhoods[:len(venue_counts)]:
        for i, ne in enumerate(neighborhoods[:len(venue_counts)]):
            if ne == neighborhood:
                idx = i
        if neighborhood not in neigh_:
            label = '{}, {}, {}, {}'.format(neighborhood, borough, color_by, original_score[idx])
            label = folium.Popup(label, parse_html=True)
            folium.CircleMarker(
                [lat, lng],
                radius=10,
                popup=label,
                color='blue',
                fill=True,
                fill_color='#3186cc',
                fill_opacity=original_score[idx],
                parse_html=False).add_to(map_newyork)  
            neigh_.append(neighborhood)

map_newyork